In [1]:
from blacbox import Saliency
from blacbox import RaceClassifier
from blacbox import GuidedBackPropagation
from blacbox import GCAM
import numpy as np
import torch
import cv2
import torchvision.transforms as transforms
import torch
import torchvision.models as models
import matplotlib.pyplot as plt

In [2]:
import torch.nn.functional as F

In [3]:
clf = RaceClassifier("/home/dazedtiara6667/GitHub/CV/EthinicAPI/fair_face_models/res34_fair_align_multi_7_20190809.pt")

In [4]:
class ScoreCAM(GCAM):
    def __init__(self, model, interpolate = 'bilinear', device = 'cpu'):
        super().__init__(model, interpolate, device)
        
    def retrieve_gcams(self, images, class_idx, key, colormap, apply_cmap = True):
        """
        Retrieving gcams from the images provided
        """
        gcams = []            
        # Selecting the index 
        
        for image in images:
            # Taking the images
            self.image = image.unsqueeze(0).to(self.device)
            hadmard_products = []
            self.image.requires_grad = True
            output = self.model(self.image)
            output = F.softmax(output)
            if(class_idx == "keepmax"):
                class_idx = output.argmax(dim = 1)
            elif(class_idx == "keepmin"):
                class_idx = output.argmin(dim = 1)

            # Calculating gradients w.r.t to the idx selected
            cr = output[0,class_idx]
            cr.backward()
                

            # Retrieving gradients and activation maps
            fmaps = self.activations[key]
            with torch.no_grad():
                for k in range(fmaps.shape[1]):
                    k_map = fmaps[:,k,:,:]
                    if k_map.max() == k_map.min():
                        continue
                    k_map = k_map.unsqueeze(0)
                    #print(k_map.shape)
                    k_map = F.interpolate(
                        k_map, self.image.shape[2:4], mode=self.interpolate, align_corners=False
                    )
                    k_map = k_map - k_map.min()/(k_map.max() - k_map.min())
                    #print(k_map.shape)
                    #print(self.image.shape)
                    hadmard_products.append(torch.mul(self.image,k_map))
                    #print(hadmard_products[-1].shape)

                hadmard_products = torch.cat(hadmard_products, dim = 0)
                print(hadmard_products.shape)
                target_scores = self.model(hadmard_products)
                print(target_scores.shape)
                cic_scores = torch.sub(target_scores[:,class_idx], cr)
                alpha = F.softmax(cic_scores)
                score_cam = torch.mul(self.activations[key], alpha).sum(dim = 1, keepdim = True)
                print(score_cam.shape)
                #print(alpha.shape)
                #print(self.activations[key].shape)
                #score_cam = F.relu(alpha*self.activations[key]).sum(dim = 1, keepdim = True)
                score_cam = F.interpolate(
                    score_cam, self.image.shape[2:4], mode=self.interpolate, align_corners=False
                )
                print(score_cam.shape)

                score_cam = score_cam.cpu().squeeze(0).detach().permute(1,2,0).numpy()
                score_cam = cv2.normalize(
                    score_cam, 
                    None,
                    alpha = 0,
                    beta = 255,
                    norm_type = cv2.NORM_MINMAX,
                    dtype = cv2.CV_8UC3
                )
                score_cam = cv2.applyColorMap(score_cam, self.colormap_dict[colormap])

                gcams.append(score_cam)
        
        # Removing the hooks, activations, gradients stored
        self.ac_handler.remove()
        self.grad_handler.remove()
        self.activations = {}
        self.gradients = {}
        return np.array(gcams)

In [5]:
model = models.resnet50(pretrained = True)

In [ ]:
cam = ScoreCAM(clf.model)

In [ ]:
cam.colormap_dict

In [ ]:
image1 = cv2.imread('blacbox/architectures/images/black.png')
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
image2 = cv2.imread('blacbox/architectures/images/dog.png')
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

In [ ]:
def preprocess_image(image):
        transform = transforms.Compose([
                        
                        transforms.ToPILImage(),
                        transforms.Resize((224, 224)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


        ])

        image = transform(image)
        return image

In [ ]:
image1 = preprocess_image(image1)
image2 = preprocess_image(image2)

In [11]:
images = torch.stack((image1, image2), axis = 0)

In [12]:
heatmaps = cam.reveal(images = images, module = clf.model.layer4[0].conv1, class_idx = 'keepmax', colormap = 'bone')

torch.Size([512, 3, 224, 224])
torch.Size([512, 18])


RuntimeError: The size of tensor a (7) must match the size of tensor b (512) at non-singleton dimension 2

In [74]:
heatmaps.shape

(2, 7, 512, 7, 7)